In [30]:
import cv2 as cv
import numpy as np
import uuid
from collections import deque

In [141]:
cap = cv.VideoCapture(1,cv.CAP_DSHOW)

lower_blue = np.array([60,50,50])
upper_blue = np.array([110,255,255])
k = np.ones((5,5),np.uint8)
pts = deque(maxlen=64)
image_final = np.full((480,640,3),(0,0,0),np.uint8)
image_linha = np.full((480,640,3),(0,0,0),np.uint8)
seed = uuid.uuid1()
fourcc = cv.VideoWriter_fourcc(*'XVID')
video = cv.VideoWriter('video_'+str(seed)+'.avi', fourcc, 20.0, (1280,960))
while True:
        isCap,frame = cap.read()

        if isCap:
            hsv = cv.cvtColor(frame,cv.COLOR_BGR2HSV)
            mask = cv.inRange(hsv, lower_blue, upper_blue)
            res = cv.bitwise_and(frame,frame, mask= mask) 
            res[np.where(res != 0)] = 255
            res = cv.medianBlur(res,7)
            res = cv.dilate(res,k,iterations = 1)    
            res = cv.cvtColor(res,cv.COLOR_HSV2BGR)
            res = cv.cvtColor(res,cv.COLOR_BGR2GRAY)

            contours,hierarchy = cv.findContours(res, cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
            res = cv.cvtColor(res,cv.COLOR_GRAY2BGR)
            
            
            if len(contours) > 0:
                cnt = contours
                c = max(cnt, key=cv.contourArea)
                ((x, y), radius) = cv.minEnclosingCircle(c)
                M = cv.moments(c)
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                cv.circle(image_final, (int(x), int(y)), int(1),(0, 255, 0), -1)
                pts.appendleft(center)
                for i in range(1, len(pts)):
                    if pts[i - 1] is None or pts[i] is None:
                        continue
                    thickness = int(np.sqrt(64 / float(i + 1)) * 1)
                    cv.line(image_linha, pts[i - 1], pts[i], (0, 0, 255), 1)
            
            
            image_cima = cv.hconcat([frame,res])
            image_baixo = cv.hconcat([image_final,image_linha])
            
            image_total = cv.vconcat([image_cima,image_baixo])
            video.write(image_total)
            
            #cv.imshow('Pendulo',image_final)
            #cv.imshow('Original',frame)
            #cv.imshow('HSV',res)
            #cv.imshow("Resultado", image_linha)
            
            cv.imshow('Resultao',image_total)

        key = cv.waitKey(30)

        if key == ord('q'):
            break
        elif key == ord('s'):
            cv.imwrite('pontos_'+str(seed)+'.jpg',image_final)
            cv.imwrite('linhas_'+str(seed)+'.jpg',image_linha)
        elif key == ord('c'):
            image_final = np.full((480,640,3),(0,0,0),np.uint8)
            image_linha = np.full((480,640,3),(0,0,0),np.uint8)
            pts.clear()

cv.destroyAllWindows()
cap.release()
video.release()

In [125]:
cv.destroyAllWindows()
cap.release()